In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import timedelta
import functions.common as cf

In [2]:
datafiles = pd.read_csv('data_files_list.csv')
reviewfiles = pd.read_csv('define_datareview_list.csv')

pd.set_option('display.max_colwidth', -1)
(datafiles)

,Unnamed: 0,datasets,method
0,deployment0001,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174413-CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered/deployment0001_CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered_20140415T132159.798580-20140706T212524.736690.nc,recovered_host
1,deployment0002,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174413-CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered/deployment0002_CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered_20150509T113514.756870-20150625T150702.264890.nc,recovered_host
2,deployment0003,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174413-CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered/deployment0003_CP05MOAS-GL379-03-CTDGVM000-recovered_host-ctdgv_m_glider_instrument_recovered_20160121T150043.663730-20160518T015328.144100.nc,recovered_host
3,deployment0001,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174435-CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument/deployment0001_CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument_20140415T134258.674740-20140706T212530.867710.nc,telemetered
4,deployment0002,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174435-CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument/deployment0002_CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument_20150509T125849.226410-20150625T123743.327300.nc,telemetered
5,deployment0003,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174435-CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument/deployment0003_CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument_20160121T150043.663730-20160518T015328.144100.nc,telemetered
6,deployment0004,https://opendap.oceanobservatories.org/thredds/dodsC/ooi/leila.ocean@gmail.com/20190306T174435-CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument/deployment0004_CP05MOAS-GL379-03-CTDGVM000-telemetered-ctdgv_m_glider_instrument_20170116T125355.760380-20170116T221950.330870.nc,telemetered


In [3]:
deploy_list = pd.unique(datafiles['Unnamed: 0'])

In [4]:
d_info = pd.DataFrame()
d_dia = pd.DataFrame()
for item in deploy_list: 
    datalist = datafiles['datasets'][datafiles['Unnamed: 0'] == item]
    method = datafiles['method'][datafiles['Unnamed: 0'] == item]
    
    if len(datalist) == 1:
        note = 'No comparison: Data exist from only 1 delivery method.'
    elif len(datalist) > 1 & len(datalist) <= 3:
        note = 'Do comparison: Data exist from 2 different delivery methods.'
    else:
        note = 'No comparison: Data exist for more than 3 delivery methods. Please provide fewer datasets for analysis.'
    
    umethods = [u.split('/')[-2].split('-')[-2] for u in datalist]
    ustreams = [u.split('/')[-2].split('-')[-1] for u in datalist]
    
    # check stream names
    omit_word = ['_dcl', '_imodem', '_conc']   
    ss_new = []
    
    for s in ustreams:
        wordi = []
        for word in omit_word:
            if word in s:
                wordi.append(word)
                break

        if wordi:
            fix = s.split(wordi[0])
            if len(fix) == 2:
                ss_new.append(fix[0] + fix[1].split('_recovered')[0])
        elif '_recovered' in s:
            ss_new.append(s.split('_recovered')[0])

        else:
            ss_new.append(s)
           
    d0 = pd.DataFrame({'note': [note], 'methods': [umethods],'streams':[ss_new], 'datasets': [datalist.values]}, index=[item])
    d_info = d_info.append(d0)
    
d_dia = d_dia.append(d0['note']) #, index=[d0.index.values]
pd.set_option('display.max_colwidth', -1)
(d_dia)

,deployment0004
note,No comparison: Data exist from only 1 delivery method.


In [5]:
df = pd.DataFrame()
df_sum =  pd.DataFrame()
for ii in range(len(d_info)): 
    index_i = d_info.index.values[ii]
    print(index_i)
    if len(d_info['methods'].values[ii]) == 2:
        ds0 = xr.open_dataset(d_info['datasets'].values[ii][0])
        ds0 = ds0.swap_dims({'obs': 'time'})        
        ds0_sci_vars = cf.return_science_vars(d_info['streams'].values[ii][0])
        ds0_method = d_info['methods'].values[ii][0]
        
        ds1 = xr.open_dataset(d_info['datasets'].values[ii][1])
        ds1 = ds1.swap_dims({'obs': 'time'})        
        ds1_sci_vars = cf.return_science_vars(d_info['streams'].values[ii][1])
        ds1_method = d_info['methods'].values[ii][1]
        
        # define preffered method
        if reviewfiles['method'].values[ii] == ds0_method:
            preferred_method = 'ds0'
        elif reviewfiles['method'].values[ii] == ds1_method: 
            preferred_method = 'ds1'
                
        # find where the variable long names are the same
        ds0names = cf.long_names(ds0, ds0_sci_vars)
        ds0names.rename(columns={'name': 'name_ds0'}, inplace=True)
        
        ds1names = cf.long_names(ds1, ds1_sci_vars)
        ds1names.rename(columns={'name': 'name_ds1'}, inplace=True)
        
        mapping = pd.merge(ds0names, ds1names, on='long_name', how='inner')
        
        df, missing_data_list, diff_gzero_list, var_list = cf.compare_datasets(df, mapping, preferred_method, index_i,ds0, ds0_method, ds1, ds1_method)
    else:
        print(d_info['note'].values[ii])

    fd_test = cf.found_data_in_another_stream(missing_data_list)
    comparison_details = cf.found_data_in_another_stream_diff(fd_test, diff_gzero_list, var_list)
    
    df_sum0 = pd.DataFrame({'data_comparison': [fd_test],
                            'comparison_details': [comparison_details]}
                           , index= [d_info.index.values[ii]])
    df_sum = df_sum.append(df_sum0)
            

deployment0001
practical_salinity Practical Salinity practical_salinity
sci_water_cond Seawater Conductivity sci_water_cond
sci_water_pressure Seawater Pressure sci_water_pressure
sci_water_pressure Seawater Pressure sci_water_pressure_dbar
sci_water_pressure_dbar Seawater Pressure sci_water_pressure
sci_water_pressure_dbar Seawater Pressure sci_water_pressure_dbar
sci_water_temp Seawater Temperature sci_water_temp
sci_seawater_density Seawater Density sci_seawater_density
deployment0002
practical_salinity Practical Salinity practical_salinity
sci_water_cond Seawater Conductivity sci_water_cond
sci_water_pressure Seawater Pressure sci_water_pressure
sci_water_pressure Seawater Pressure sci_water_pressure_dbar
sci_water_pressure_dbar Seawater Pressure sci_water_pressure
sci_water_pressure_dbar Seawater Pressure sci_water_pressure_dbar
sci_water_temp Seawater Temperature sci_water_temp
sci_seawater_density Seawater Density sci_seawater_density
deployment0003
practical_salinity Practical 

In [6]:
df = df[df['unit_test'] != 'fail']
df = df[df['unit'] != 'bar']

In [7]:
pd.set_option('display.max_colwidth', -1)
(df)

,name,unit,unit_test,n,n_nan,missing_data,n_comparison,min_abs_diff,max_abs_diff,n_diff_greater_zero,percent_diff_greater_zero
deployment0001,practical_salinity,1,pass,1697681,4,no missing data,52877,0.0,0.000000,0,0.00
deployment0001,sci_water_cond,S m-1,pass,1697681,0,no missing data,52924,0.0,0.000000,0,0.00
deployment0001,sci_water_pressure_dbar,dbar,pass,1697681,0,no missing data,52940,0.0,0.000000,0,0.00
deployment0001,sci_water_temp,ºC,pass,1697681,0,no missing data,52932,0.0,0.000000,0,0.00
deployment0001,sci_seawater_density,kg m-3,pass,1697681,4,no missing data,52877,0.0,0.000007,0,0.00
deployment0002,practical_salinity,1,pass,1876498,9,no missing data,32316,0.0,0.000000,0,0.00
deployment0002,sci_water_cond,S m-1,pass,1876498,0,no missing data,32534,0.0,0.000000,0,0.00
deployment0002,sci_water_pressure_dbar,dbar,pass,1876498,0,no missing data,32548,0.0,0.000000,0,0.00
deployment0002,sci_water_temp,ºC,pass,1876498,0,no missing data,32550,0.0,0.000000,0,0.00
deployment0002,sci_seawater_density,kg m-3,pass,1876498,9,no missing data,32316,0.0,0.000009,0,0.00


In [9]:
pd.set_option('display.max_colwidth', -1)
(df_sum)

,data_comparison,comparison_details
deployment0001,pass,"{'99': 6, '0': 2}"
deployment0002,pass,"{'99': 6, '0': 2}"
deployment0003,pass,"{'99': 6, '0': 2}"
deployment0004,pass,"{'99': 6, '0': 2}"
